In [2]:
!nvidia-smi

Sat Nov 15 07:34:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [4]:
# Load Model and tokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map = "cuda", 
    torch_dtype = "auto", 
    trust_remote_code = True, 
)

# Creata a pipeline

generator = pipeline(
    "text-generation", 
    model = model, 
    tokenizer = tokenizer,
    return_full_text = False, 
    max_new_tokens = 50, 
    do_sample = False
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [7]:
prompt = "Write an email apologizing to sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt, use_cache = False)

print(output[0]["generated_text"])



Dear Sarah,

I am deeply sorry for the unfortunate incident that occurred in my garden. I understand that it must have been quite distressing for you to witness the damage caused to your beloved plants.


